# Guia Abrangente de Regressão Linear Multinível com PyMC

Este guia serve como um manual prático e conceitual para a construção, avaliação e utilização de modelos de regressão linear multinível (ou hierárquicos) usando a biblioteca PyMC. Usaremos o dataset *California Housing* como nosso estudo de caso, seguindo a estrutura do código fornecido, mas aprofundando cada passo.

## O que é um Modelo Multinível e Por Que Usá-lo?

Imagine que você quer prever o desempenho de alunos em um país. Você poderia criar um único modelo de regressão para todos os alunos. No entanto, é razoável supor que alunos da mesma escola são mais parecidos entre si do que com alunos de outras escolas, devido a professores, recursos e localização em comum.

Um modelo de regressão simples ignora essa estrutura de "agrupamento" (alunos dentro de escolas). Ele trata cada aluno como uma observação independente, o que não é verdade.

**É aqui que os modelos multinível brilham.** Eles permitem que os parâmetros do seu modelo (como o intercepto e os coeficientes) variem entre os grupos. Em vez de calcular um único intercepto para todos os alunos, calculamos um intercepto para cada escola. O modelo, então, aprende a "puxar" (ou "encolher", do inglês *shrinkage*) os interceptos das escolas em direção à média geral, especialmente para escolas com poucos dados.

**Vantagens:**
1.  **Reconhece a Estrutura dos Dados:** Modela explicitamente a não independência das observações dentro dos grupos.
2.  **Melhora as Estimativas:** As estimativas para cada grupo se beneficiam da informação de todos os outros grupos (conceito de *pooling* parcial).
3.  **Evita Overfitting:** O *shrinkage* previne que o modelo se ajuste demais a ruídos em grupos pequenos.
4.  **Flexibilidade:** Permite que tanto o ponto de partida (intercepto) quanto a relação entre variáveis (inclinação/coeficiente) mudem por grupo.

No nosso caso do dataset da Califórnia, as "escolas" serão regiões geográficas que criaremos. É intuitivo pensar que a relação entre a renda média e o preço dos imóveis não é a mesma em Beverly Hills e em uma área rural do interior.

---

## Parte 1: Preparação e Exploração dos Dados

O primeiro passo de qualquer projeto de modelagem é entender os dados.

### 1.1. Carregando e Inspecionando os Dados

In [1]:
import pandas as pd
import pymc as pm
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import arviz as az

# Carrega os dados de treino
df = pd.read_csv("sample_data/california_housing_train.csv")

# Extrai as variáveis de interesse para arrays numpy
median_income = df['median_income'].values
housing_median_age = df['housing_median_age'].values
median_house_value = df['median_house_value'].values
population = df['population'].values

**Como Pensar Sobre Isso:**
*   **O que fazer:** Sempre comece carregando os dados e dando uma olhada inicial com `df.head()` e `df.describe()`. Isso te dá uma noção da escala das variáveis, se há valores faltantes, etc.
*   **Por que fazer:** Extrair as colunas para arrays NumPy, como feito no script, é uma boa prática para usar com PyMC, pois ele funciona de forma mais eficiente com arrays NumPy do que com Series do Pandas.

### 1.2. Exploração Geoespacial

Como nossos dados possuem latitude e longitude, a visualização em um mapa é extremamente valiosa.

In [2]:
# Converte o DataFrame do Pandas para um GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
gdf.crs = "EPSG:4326" # Define o Sistema de Coordenadas (WGS 84, padrão para GPS)

# Visualização com lets_plot (uma alternativa poderosa ao matplotlib)
from lets_plot import *
LetsPlot.setup_html()

data = {'lon': gdf.geometry.x, 'lat': gdf.geometry.y, 'value': gdf['median_house_value']}

p = ggplot(data) + \
    geom_livemap() + \
    geom_point(aes(x='lon', y='lat', size='value', color='value'), alpha=0.5) + \
    scale_color_gradient(low='blue', high='red') + \
    ggtitle('Valor Mediano das Casas na Califórnia')

p.show()

ModuleNotFoundError: No module named 'lets_plot'